In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model, Sequential, load_model, save_model
from tensorflow.keras.optimizers import SGD, Adam
from keras.preprocessing.image import load_img, img_to_array
import os
from os import listdir
from numpy import save
from keras.models import load_model
import sys
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from matplotlib.image import imread
from shutil import copyfile
from random import seed, random
from Resnet import Resnet
import tkinter as tk
from tkinter import filedialog
from PIL import Image

In [ ]:
data_folder = 'IMG_CLASSES/'
database_home = 'dataset/'

In [ ]:
def data_processing():
    if not os.path.exists(database_home):
        os.makedirs(database_home)
    
    photos, labels = list(), list()
    
    for folder_name in listdir(data_folder):
        folder_path = os.path.join(data_folder, folder_name)
        if os.path.isdir(folder_path):  # Corrected to use folder_path
            if folder_name.startswith('1. Eczema'):
                output = 0.0
            elif folder_name.startswith('2. Melanoma'):
                output = 1.0
            elif folder_name.startswith('3. Atopic Dermatitis'):
                output = 2.0
            elif folder_name.startswith('4. Basel Cell Carcinoma'):
                output = 3.0
            elif folder_name.startswith('5. Melanocytic Nevi'):
                output = 4.0
            elif folder_name.startswith('6. Benign Keratosis-like Lesions (BKL)'):
                output = 5.0
            elif folder_name.startswith('7. Psoriasis Lichen Planus'):
                output = 6.0
            elif folder_name.startswith('8. Seborrheic Keratoses'):
                output = 7.0
            elif folder_name.startswith('9. Tinea Ringworm Candidiasis'):
                output = 8.0
            elif folder_name.startswith('10. Warts Molluscum'):
                output = 9.0
            
            for filename in listdir(folder_path):
                image_path = os.path.join(folder_path, filename)
                # Load photos
                photo = load_img(image_path, target_size=(224, 224))
                # Convert to numpy array
                photo = img_to_array(photo)
                photos.append(photo)
                labels.append(output)

    # Convert lists to numpy arrays after the loop
    photos = np.asarray(photos)
    labels = np.asarray(labels)
    print(photos.shape, labels.shape)
    
    # Save arrays
    save('photos.npy', photos)
    save('labels.npy', labels)
    
    subdirs = ['train/', 'test/']
    for subdir in subdirs:
        labeldirs = ['Eczema/', 'Melanoma/', 'Atopic Dermatitis/', 
                     'Basel Cell Carcinoma/', 'Melanocytic Nevi/', 
                     'Benign Keratosis-like Lesions (BKL)/', 'Psoriasis Lichen Planus/', 
                     'Seborrheic Keratoses/', 'Tinea Ringworm Candidiasis/', 'Warts Molluscum/']
        for labldir in labeldirs:
            newdir = os.path.join(database_home, subdir, labldir)
            if not os.path.exists(newdir):
                os.makedirs(newdir)

            
            
    

In [ ]:
def insert_data():
    seed(1)
    val_ratio = 0.25
    #copy data_folder images
    for folder in listdir(data_folder):
        folder_path = os.path.join(data_folder, folder)
        print(f'Check folder: {folder_path}')
        if os.path.isdir(folder_path):
            print(f'Processing: {folder}')
            for filename in listdir(folder_path):
                src = os.path.join(folder_path, filename)
                dst_dir = 'train/'
                if random() < val_ratio:
                    dst_dir = 'test/'
                dst = None
                if 'Eczema' in folder:
                    class_name = 'Eczema'
                elif 'Melanoma' in folder:
                    class_name = 'Melanoma'
                elif 'Atopic Dermatitis' in folder:
                    class_name = 'Atopic Dermatitis'
                elif 'Basal Cell Carcinoma' in folder:
                    class_name = 'Basal Cell Carcinoma'
                elif 'Melanocytic Nevi' in folder:
                    class_name = 'Melanocytic Nevi'
                elif 'Benign Keratosis-like Lesions (BKL)' in folder:
                    class_name = 'Benign Keratosis-like Lesions (BKL)'
                elif 'Psoriasis Lichen Planus' in folder:
                    class_name = 'Psoriasis Lichen Planus'
                elif 'Seborrheic Keratoses' in folder:
                    class_name = 'Seborrheic Keratoses'
                elif 'Tinea Ringworm Candidiasis' in folder:
                    class_name = 'Tinea Ringworm Candidiasis'
                elif 'Warts Molluscum' in folder:
                    class_name = 'Warts Molluscum'              
                dst = os.path.join(database_home, dst_dir, class_name, filename)
                if dst:
                    print(f"Copying {src} to {dst}")
                    try:
                        os.makedirs(os.path.dirname(dst), exist_ok=True)
                        copyfile(src, dst)
                    except Exception as e:
                        print(f"Failed to copy {src} to {dst}: {e}")
            
            
            

In [ ]:
def define_model():
    model = Resnet()
    opt = Adam()
    model.compile(optimizer=opt, loss='crossentropy', metrics=['accuracy'])
    return model

In [ ]:
#plot diagnostic learning curves
def summarize_diagnostics(histories):
    #plot loss
    plt.subplot(1, 2, 1)
    plt.title('Cross Entropy Loss')
    plt.plot(histories.history['loss'], color='blue', label='train')
    plt.plot(histories.history['val_loss'], color='orange', label='test')
    #plot accuracy
    plt.subplot(1, 2, 2)
    plt.title('Classification Accuracy')
    plt.plot(histories.history['accuracy'], color='blue', label='train')
    plt.plot(histories.history['val_accuracy'], color='orange', label='test')
    plt.show()
    #save plot to file
    filename = sys.argv[0].split('/')[-1]
    plt.savefig(filename + '_plot.png')
    plt.close()

In [ ]:
def run_test_harness():
    #data_processing()
    #insert_data()
    train_datagen = ImageDataGenerator(rescale=1.0/255.0, width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True)
    test_datagen = ImageDataGenerator(rescale=1.0/255.0)
    train_it = train_datagen.flow_from_directory(f'{database_home}train/', class_mode='categorical', batch_size= 128, target_size=(224,224))
    test_it = test_datagen.flow_from_directory(f'{database_home}test/', class_mode='categorical', batch_size=128, target_size=(224,224))
    #load model
    filename = 'model.keras'
    if os.path.isfile(filename):
        model = load_model(filename)
        # Compile model with the desired optimizer and loss function
        opt = Adam()
        model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    else:
        #define model
        model = define_model()
    #fit model
    history = model.fit(train_it, 
                        steps_per_epoch=(train_it.samples // train_it.batch_size), 
                        validation_data=test_it, 
                        validation_steps=(test_it.samples // test_it.batch_size), 
                        epochs=50, verbose=1)
    #evaluate model
    _, acc = model.evaluate(test_it, 
                            steps=(test_it.samples // test_it.batch_size), 
                            verbose=0) 
    print('> %.3f' % (acc * 100.0))
    #learning curves
    summarize_diagnostics(history)
    #save model
    model.save(filename)

In [ ]:
run_test_harness()

In [ ]:
def load_image(filename):
    img = load_img(filename, target_size=(224, 224))
    img = img_to_array(img)
    img = img.reshape(1, 224, 224, 3)
    img = img.astype('float32')
    img -= [123.68, 116.779, 103.939]
    return img

In [ ]:
model_file = 'model.keras'
model_name = Resnet()

In [ ]:
def recreate_model():
    return model_name

In [ ]:
def open_image():
    root = tk.Tk()
    root.withdraw()
    file_path = filedialog.askopenfilename(
        title="Select an Image",
        filetypes=[("Image Files", "*.jpg;*.jpeg;*.png;*.bmp")]
    )
    return file_path

In [ ]:
def run_model():
    file = open_image()
    if file:
        print(f'File: {file} selected')
        model = load_model(model_file, custom_objects={f'{model_name}': recreate_model()})
        img = load_image(file)
        pred = model.predict(img)
        result = np.argmax(pred, axis=1)
        return result[0]
    else:
        print('No file selected')

In [ ]:
result = run_model()
diseases = ['Eczema', 'Melanoma', 'Atopic Dermatitis', 'Basel Cell Carcinoma',
                    'Melanocytic Nevi', 'Benign Keratosis-like Lesions (BKL)', 
                    'Psoriasis Lichen Planus', 'Seborrheic Keratoses', 
                    'Tinea Ringworm Candidiasis', 'Warts Molluscum']
if result == 0.0:
    print(f'You need to make an appointment with a doctor because you have a high chance of having {diseases[0]} and need cortocosteroids and you should buy treatment cream')
elif result == 1.0:
    print(f'This is {diseases[1]} (which is skin cancer). Please visit the closest hospital to examine the sign appears on your skin before it gets worse!')
elif result == 2.0:
    print(f"This is {diseases[2]}, you may start with regular moisturizing and other self-care habits. If these don't help, your health care provider might suggest medicated creams that control itching and help repair skin. These are sometimes combined with other treatments.")
elif result == 3.0:
    print(f"This is {diseases[3]}, it is not life-threatening and does not spread, but can grow larger if left untreated. Please make an appointment with doctor and buy Hydroquinone Cream to have on your skin!")
elif result == 4.0:
    print(f"No worries, this is called {diseases[4]}. Please buy Imiquimod 5% Cream!")
elif result == 5.0:
    print(f"You got {diseases[5]}. It won’t require any treatment. But you might want to have it removed if it becomes itchy or irritated or you don’t like the look of it. Your healthcare provider can remove it for you in the office using one of several common methods.")
elif result == 6.0:
    print(f"You got {diseases[6]}. These following treatments may help relieve your symptoms: Corticosteroid creams, Antihistamines, Phototherapy, and Immunosuppressants.")
elif result == 7.0:
    print(f"You got {diseases[7]}. You should visit nearest medical office to receive the following treatments like: Cryotherapy, Electrodessication, Shave Excision, or Laser Therapy.")
elif result == 8.0:
    print(f"You got {diseases[8]}. You should buy some non-prescription like: Clotrimazole, Miconazole, Terbinafine, or Ketoconazole. If it got worse, please contact a healthcare provider!")
elif result == 9.0:
    print(f"You got {diseases[9]}. You should visit a healthcare provider for further examination. You should also buy the following unprescribed treatment: Imiquimod cream.")